In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

Next we need to import the kwik file to analyze spike

In [3]:
hdf5_file_name = 'openephys.kwik'
data_set_name = '/channel_groups/0/spikes/clusters/main/1/cluster_group' #where is located the data you want inside the hdf5 file tree

file    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
#data = file[data_set_name]

# see file tree
def print_attrs(name, obj):
    print name
    for key, val in obj.attrs.iteritems():
        print "    %s: %s" % (key, val)

file.visititems(print_attrs)


application_data
application_data/klustakwik2
    num_starting_clusters: 200
    max_possible_clusters: 400
application_data/spikedetekt
    threshold_strong_std_factor: 6
    use_single_threshold: 0
    filter_low: 500.0
    dtype: ['int16']
    threshold_weak_std_factor: 3.0
    n_channels: 32
    filter_lfp_low: 0
    raw_data_files: ['openephys.dat']
    chunk_size_seconds: 1
    filter_high_factor: 0.475
    prb_file: ['1x32_buzsaki']
    voltage_gain: 10.0
    extract_s_after: 16
    detect_spikes: ['negative']
    n_excerpts: 50
    filter_butter_order: 3
    weight_power: 2
    excerpt_size_seconds: 1
    connected_component_join_size: 1
    extract_s_before: 16
    experiment_name: ['openephys.kwik']
    sample_rate: 30000
    n_features_per_channel: 3
    chunk_overlap_seconds: 0.015
    pca_n_waveforms_max: 10000
    filter_lfp_high: 300
channel_groups
channel_groups/0
    channel_order: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28

    color: 3
channel_groups/0/clusters/main/143
    cluster_group: 0
channel_groups/0/clusters/main/143/application_data
channel_groups/0/clusters/main/143/application_data/klustaviewa
    color: 9
channel_groups/0/clusters/main/144
    cluster_group: 0
channel_groups/0/clusters/main/144/application_data
channel_groups/0/clusters/main/144/application_data/klustaviewa
    color: 4
channel_groups/0/clusters/main/145
    cluster_group: 0
channel_groups/0/clusters/main/145/application_data
channel_groups/0/clusters/main/145/application_data/klustaviewa
    color: 8
channel_groups/0/clusters/main/146
    cluster_group: 0
channel_groups/0/clusters/main/146/application_data
channel_groups/0/clusters/main/146/application_data/klustaviewa
    color: 8
channel_groups/0/clusters/main/147
    cluster_group: 0
channel_groups/0/clusters/main/147/application_data
channel_groups/0/clusters/main/147/application_data/klustaviewa
    color: 5
channel_groups/0/clusters/main/148
    cluster_group: 0
channe

channel_groups/0/clusters/original/104
    cluster_group: 3
channel_groups/0/clusters/original/104/application_data
channel_groups/0/clusters/original/104/application_data/klustaviewa
    color: 2
channel_groups/0/clusters/original/105
    cluster_group: 3
channel_groups/0/clusters/original/105/application_data
channel_groups/0/clusters/original/105/application_data/klustaviewa
    color: 4
channel_groups/0/clusters/original/106
    cluster_group: 3
channel_groups/0/clusters/original/106/application_data
channel_groups/0/clusters/original/106/application_data/klustaviewa
    color: 5
channel_groups/0/clusters/original/107
    cluster_group: 3
channel_groups/0/clusters/original/107/application_data
channel_groups/0/clusters/original/107/application_data/klustaviewa
    color: 4
channel_groups/0/clusters/original/108
    cluster_group: 3
channel_groups/0/clusters/original/108/application_data
channel_groups/0/clusters/original/108/application_data/klustaviewa
    color: 6
channel_groups/

In [5]:
auxGood = file['/channel_groups/0/clusters/main/']
#a.visititems(print_attrs)
#a.attrs['cluster_group']

c = 0;
tag = np.zeros(np.size(np.concatenate((range(1),range(2,np.size(auxGood)+1)))))
for cls in np.concatenate((range(1),range(2,np.size(auxGood)+1))):
    aux = file['/channel_groups/0/clusters/main/' + str(cls)]
    tag[c] = aux.attrs['cluster_group'] #cluster tag as described below for 1 and 2
    c = c+1
#1 is MUA (needs to be cleaned in a manual way, we'll have to build an interface to do it)
#2 is GOOD clusters, just extract and analyze

#/channel_groups/0/clusters/main/<cluster_number>/cluster_group
aux = np.where(tag >= 1); #need to sum 1 to retrieve correct clusters

GoodMUAclus = aux[0]+1;
GoodMUAclus

array([  3,   5,   7,  11,  12,  15,  16,  18,  23,  25,  30,  31,  32,
        36,  38,  97,  99, 105, 109, 112, 137, 140, 141, 157, 167, 170,
       177, 181, 184, 188, 190, 192])

In [6]:
auxCluster = file['/channel_groups/0/spikes/clusters/main']
#auxChannel.visititems(print_attrs)
# '/channel_groups/0/spikes/time_samples' address to get sample time for each spike
# '/channel_groups/0/spikes/clusters/main' addres to get cluster for each spike
# 
np.shape(auxCluster)
#auxChannel.attrs['features_masks']

(3564461,)

In [8]:
hdf5_file_name = 'openephys.kwx'
data_set_name = '/channel_groups/0/spikes/clusters/main/1/cluster_group' #where is located the data you want inside the hdf5 file tree

arquiv    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
selectChannel = arquiv['/channel_groups/0/features_masks']
np.shape(selectChannel)
#selectChannel[23,:,1] #the second dimension is indexing channel first and features laters, as it was said in the google groups, but some exploring would be good


(3564461, 96, 2)

Separating to which channel a spike should be

In [9]:
#Changin arrays to numpy arrays to work better
auxCluster = np.array(auxCluster)
GoodMUAclus = np.array(GoodMUAclus)

#create a dictionare to save the spikes for each clsuter
from collections import defaultdict
Neurons = defaultdict(list)

for cls in range(np.size(GoodMUAclus)):
    Neurons[GoodMUAclus[cls]]
    
for idx in range(np.size(GoodMUAclus)):
        aux = np.where(auxCluster == GoodMUAclus[idx]) #selecting which spike is in good clusters
        Neurons[GoodMUAclus[idx]] = aux[0]
        
        #save in a python structure which firing goes to each clsuter
        #if Neurons[aux2[0]] == []:
        #    Neurons[aux2[0]] = np.array(idx)
        #else:
        #    Neurons[aux2[0]] = np.array([Neurons[aux2[0]],idx])
        
Neurons

defaultdict(list,
            {3: array([   1186,    2153,    8509, ..., 3531748, 3531821, 3531955]),
             5: array([      1,       9,      24, ..., 3564441, 3564445, 3564456]),
             7: array([     14,      56,      77, ..., 3564328, 3564356, 3564435]),
             11: array([     11,      12,      37, ..., 3563973, 3564091, 3564277]),
             12: array([1369613, 1486548, 1518897, ..., 3036127, 3038049, 3056324]),
             15: array([     20,      27,      45, ..., 3564433, 3564447, 3564455]),
             16: array([     90,     200,     254, ..., 3524173, 3527382, 3554807]),
             18: array([      8,      17,      41, ..., 3564416, 3564423, 3564450]),
             23: array([    159,     210,     240, ..., 3564104, 3564443, 3564444]),
             25: array([     70,      96,     248, ..., 3564314, 3564326, 3564459]),
             30: array([     18,      42,      59, ..., 3564377, 3564385, 3564410]),
             31: array([   4197,    4830,    4836,

Now I should use Neurons dict to get timestamps and channel. I should change the key from dict to the channel value, not cluster number

In [10]:
auxTimeStamps = file['/channel_groups/0/spikes/time_samples'] #getting the time sample for the time stamps
NeuronTS = defaultdict(list)
AllTS = range(np.size(NeuronTS))
for cls in range(np.size(GoodMUAclus)):
    aux = Neurons[GoodMUAclus[cls]][:]
    auxArr = np.zeros(np.size(aux))#array auxiliar
    for idx in range(np.size(aux)):
        auxArr[idx] = auxTimeStamps[aux[idx]]
    NeuronTS[GoodMUAclus[cls]] = auxArr
    
NeuronTS

defaultdict(list,
            {3: array([  1.15652000e+05,   2.01429000e+05,   8.55533000e+05, ...,
                      3.58789964e+08,   3.58798339e+08,   3.58813379e+08]),
             5: array([  2.99000000e+02,   2.56300000e+03,   4.68100000e+03, ...,
                      3.63189824e+08,   3.63190497e+08,   3.63191089e+08]),
             7: array([  3.50700000e+03,   8.81200000e+03,   1.18690000e+04, ...,
                      3.63175287e+08,   3.63179391e+08,   3.63188969e+08]),
             11: array([  2.72700000e+03,   3.21900000e+03,   5.88000000e+03, ...,
                      3.63134438e+08,   3.63148088e+08,   3.63167362e+08]),
             12: array([  1.24268429e+08,   1.32911848e+08,   1.35766517e+08, ...,
                      2.93236858e+08,   2.93338539e+08,   2.95623053e+08]),
             15: array([  4.13200000e+03,   4.85100000e+03,   7.81800000e+03, ...,
                      3.63188880e+08,   3.63190552e+08,   3.63191012e+08]),
             16: array([  1.317

For next we should save the channel to which one pertains and save the dict to use for analysis

In [11]:
Channels = np.zeros(np.size(GoodMUAclus))
for cls in range(np.size(GoodMUAclus)):
    aux = Neurons[GoodMUAclus[cls]][:]
    auxArr = np.zeros(np.size(aux))#array auxiliar
    #temp = np.zeros(32)
    for idx in range(np.size(aux)):
        for ch in range(32):
            temp = np.sum(selectChannel[aux[idx],(3*ch):(3*ch)+3,1]) #three features per channel
            if (temp == 3):
                auxArr[idx] = ch
                break
    auxCh = stats.mode(auxArr)
    Channels[cls] = auxCh[0][0]
    
Channels
    
#selectChannel[spikenumber,feature,1]

array([ 31.,  11.,  29.,  28.,   6.,  27.,  19.,  21.,  19.,  28.,  10.,
        22.,  30.,  18.,  20.,  31.,  29.,   6.,  28.,  30.,  31.,  29.,
         6.,  29.,  31.,  29.,  29.,   6.,   6.,  20.,  31.,  11.])

In [12]:
NeuronTS['Channels'] = Channels
NeuronTS['Cluster'] = GoodMUAclus

np.save('NeuronTS.npy', NeuronTS)
